In [10]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)
import pandas as pd
import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
from matplotlib import warnings
# Import the backtrader platform
import backtrader as bt
sys.path.append(os.path.abspath(os.path.join('../scripts')))
data=pd.read_csv('../data/MSFT.csv')


In [12]:
# Create a Stratey
class TestStrategy(bt.Strategy):

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] < self.dataclose[-1]:
                    # current close less than previous close

                    if self.dataclose[-1] < self.dataclose[-2]:
                        # previous close less than the previous close

                        # BUY, BUY, BUY!!! (with default parameters)
                        self.log('BUY CREATE, %.2f' % self.dataclose[0])

                        # Keep track of the created order to avoid a 2nd order
                        self.order = self.buy()

        else:

            # Already in the market ... we might sell
            if len(self) >= (self.bar_executed + 5):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(TestStrategy)

    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    # modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    # datapath = os.path.join(modpath, '../data/MSFT.csv')
    sys.path.append(os.path.abspath(os.path.join('../scripts')))
    datapath='../data/MSFT.csv'

    # Create a Data Feed
    data = bt.feeds.YahooFinanceCSVData(
        dataname=datapath,
        # Do not pass values before this date
        fromdate=datetime.datetime(2021, 1, 1),
        # Do not pass values before this date
        todate=datetime.datetime(2022, 8, 31),
        # Do not pass values after this date
        reverse=False)

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=0.001)

    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run()
    # cerebro.plot()

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2021-01-04, Close, 214.48
2021-01-04, BUY CREATE, 214.48
2021-01-05, BUY EXECUTED, Price: 214.06, Cost: 214.06, Comm 0.21
2021-01-05, Close, 214.69
2021-01-06, Close, 209.12
2021-01-07, Close, 215.07
2021-01-08, Close, 216.38
2021-01-11, Close, 214.28
2021-01-12, Close, 211.76
2021-01-12, SELL CREATE, 211.76
2021-01-13, SELL EXECUTED, Price: 210.86, Cost: 214.06, Comm 0.21
2021-01-13, OPERATION PROFIT, GROSS -3.20, NET -3.62
2021-01-13, Close, 213.15
2021-01-14, Close, 209.88
2021-01-15, Close, 209.51
2021-01-15, BUY CREATE, 209.51
2021-01-19, BUY EXECUTED, Price: 210.60, Cost: 210.60, Comm 0.21
2021-01-19, Close, 213.25
2021-01-20, Close, 221.03
2021-01-21, Close, 221.65
2021-01-22, Close, 222.62
2021-01-25, Close, 226.14
2021-01-26, Close, 228.90
2021-01-26, SELL CREATE, 228.90
2021-01-27, SELL EXECUTED, Price: 234.49, Cost: 210.60, Comm 0.23
2021-01-27, OPERATION PROFIT, GROSS 23.89, NET 23.44
2021-01-27, Close, 229.46
2021-01-28, Close, 235.41
20